In [ ]:
output_dir = "../outputs/maps"

In [ ]:
# Example: Route from Prenzlauer Allee to Bonanza Coffee Roasters

result_berlin = run_one_od_routing(
    segments_panel_gdf=segments_df,
    crossings_gdf=crossings_gdf,
    junction_panel_gdf=junction_df,
    year=2023, month=6,
    origin_lonlat=(13.4037, 52.5365),      # Prenzlauer Allee 80
    dest_lonlat=(13.4047, 52.4989),        # Bonanza Coffee Roasters
    eps=0.10,
    alpha=1.0, beta=1.0, gamma=0.2,
)

print("=" * 60)
print("ROUTE: Prenzlauer Allee 80 → Bonanza Coffee Roasters")
print("=" * 60)
print("\nGraph sanity check:")
print(result_berlin["graph_sanity"])

print("\n" + "=" * 60)
print("SHORTEST DISTANCE ROUTE")
print("=" * 60)
print(f"Length: {result_berlin['shortest_length_stats']['length_m']:.1f} m")
print(f"Risk: {result_berlin['shortest_length_stats']['seg_risk_sum']:.2f}")
print(f"Cost: {result_berlin['shortest_length_stats']['cost_sum']:.2f}")

print("\n" + "=" * 60)
print("SAFEST ROUTE (min risk, ≤ +10% distance)")
print("=" * 60)
print(f"Length: {result_berlin['constrained_min_risk_stats']['length_m']:.1f} m")
print(f"Risk: {result_berlin['constrained_min_risk_stats']['seg_risk_sum']:.2f}")
print(f"Cost: {result_berlin['constrained_min_risk_stats']['cost_sum']:.2f}")

# Calculate trade-off
extra_dist = result_berlin['constrained_min_risk_stats']['length_m'] - result_berlin['shortest_length_stats']['length_m']
risk_reduction = result_berlin['shortest_length_stats']['seg_risk_sum'] - result_berlin['constrained_min_risk_stats']['seg_risk_sum']
extra_dist_pct = (extra_dist / result_berlin['shortest_length_stats']['length_m']) * 100
risk_reduction_pct = (risk_reduction / result_berlin['shortest_length_stats']['seg_risk_sum']) * 100 if result_berlin['shortest_length_stats']['seg_risk_sum'] > 0 else 0

print("\n" + "=" * 60)
print("TRADE-OFF ANALYSIS")
print("=" * 60)
print(f"Extra distance: {extra_dist:.1f} m ({extra_dist_pct:.1f}%)")
print(f"Risk reduction: {risk_reduction:.2f} ({risk_reduction_pct:.1f}%)")

In [ ]:
def hex_color_from_risk(risk_value, min_risk=0, max_risk=None):
    """
    Convert risk value to hex color.
    Low risk → dark green (#00AA00)
    Medium risk → orange (#FFA500)
    High risk → red (#FF0000)
    """
    if max_risk is None:
        max_risk = risk_value * 2  # Fallback
    
    if max_risk == min_risk:
        normalized = 0.5
    else:
        normalized = (risk_value - min_risk) / (max_risk - min_risk)
    normalized = max(0, min(1, normalized))  # Clamp to [0, 1]
    

    # Dark Green → Orange → Red gradient (3-point)
    if normalized <= 0.5:
        # Dark Green (#00AA00) to Orange (#FFA500)
        t = normalized * 2  # 0 to 1 in green-orange transition
        r = int(255 * t)      # 0 → 255
        g = int(170 * (1 - t * 0.33))  # 170 → 112 (fade from dark green to orange)
        b = 0
    else:
        # Orange (#FFA500) to Red (#FF0000)
        t = (normalized - 0.5) * 2  # 0 to 1 in orange-red transition
        r = 255               # stays at 255
        g = int(165 * (1 - t))     # 165 → 0
        b = 0
    
    return f"#{r:02x}{g:02x}{b:02x}"


def visualize_routes_with_risk_segments(
    result,
    origin_lonlat,
    dest_lonlat,
    metric_epsg=32633,
    zoom_start=13,
    save_path=None,
    risk_scale="relative",
    segments_gdf=None,
    risk_col="risk_accidents_per_10k_trips",
):
    """
    Visualize routes with individual segments colored by risk.
    - Extracts all edges from both paths
    - Colors each edge by its seg_risk value (green → orange → red gradient)
    - Both routes visible with distinct visual styles
    - Safest path: thick dashed lines (underneath)
    - Shortest path: thinner solid lines (on top)
    
    Parameters:
    -----------
    risk_scale : str
        "relative" (default) - color scale based on min/max risk in selected paths
        "overall" - color scale based on all segments in segments_gdf
    segments_gdf : gpd.GeoDataFrame, optional
        Required when risk_scale="overall". Full segment dataset for computing global min/max risk.
    risk_col : str
        Column name in segments_gdf for risk values (default: "risk_accidents_per_10k_trips")
    """
    G = result["graph"]
    
    center_lat = (origin_lonlat[1] + dest_lonlat[1]) / 2
    center_lon = (origin_lonlat[0] + dest_lonlat[0]) / 2
    
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=zoom_start,
        tiles='OpenStreetMap'
    )
    
    def extract_segments_from_path(path_nodes, route_name):
        """Extract all edges and their data from a path"""
        segments = []
        for i in range(len(path_nodes) - 1):
            u, v = path_nodes[i], path_nodes[i+1]
            
            try:
                if isinstance(G, nx.MultiGraph):
                    edges_dict = G.get_edge_data(u, v)
                    if not edges_dict:
                        continue
                    
                    # Pick best edge by cost
                    best_key = None
                    best_cost = float('inf')
                    for key, edge_attr in edges_dict.items():
                        cost = edge_attr.get('cost', edge_attr.get('length_m', float('inf')))
                        if cost < best_cost:
                            best_cost = cost
                            best_key = key
                    
                    edge_data = edges_dict[best_key]
                else:
                    edge_data = G.get_edge_data(u, v)
                    if not edge_data:
                        continue
                
                geom = edge_data.get('geometry')
                if geom is None:
                    continue
                
                # Convert to lat/lon
                gdf_temp = gpd.GeoDataFrame(
                    {'geometry': [geom]},
                    crs=f'EPSG:{metric_epsg}'
                ).to_crs(epsg=4326)
                
                geom_latlon = gdf_temp.iloc[0].geometry
                coords = [[lat, lon] for lon, lat in geom_latlon.coords]
                
                seg_risk = float(edge_data.get('seg_risk', 0.0))
                seg_id = edge_data.get('segment_id', 'unknown')
                length_m = float(edge_data.get('length_m', 0.0))
                
                segments.append({
                    'coords': coords,
                    'risk': seg_risk,
                    'segment_id': seg_id,
                    'length_m': length_m,
                    'route': route_name
                })
            
            except Exception as e:
                print(f"Error processing edge {u}-{v}: {e}")
                continue
        
        return segments
    
    # Extract segments from both paths
    print("Extracting shortest path segments...")
    shortest_segments = extract_segments_from_path(
        result["shortest_length_path"], 
        "shortest"
    )
    print(f"✓ {len(shortest_segments)} segments in shortest path")
    
    print("Extracting safest path segments...")
    safest_segments = extract_segments_from_path(
        result["constrained_min_risk_path"], 
        "safest"
    )
    print(f"✓ {len(safest_segments)} segments in safest path")
    
    # Find min/max risk for color scaling
    if risk_scale == "overall":
        if segments_gdf is None:
            print("⚠️  risk_scale='overall' but segments_gdf not provided. Falling back to 'relative'.")
            risk_scale = "relative"
        else:
            print(f"Using OVERALL risk scale (all segments in dataset)...")
            risk_values = pd.to_numeric(segments_gdf[risk_col], errors='coerce')
            risk_values = risk_values.dropna()
            min_risk = float(risk_values.min()) if len(risk_values) > 0 else 0
            max_risk = float(risk_values.max()) if len(risk_values) > 0 else 1
    
    if risk_scale == "relative":
        print(f"Using RELATIVE risk scale (selected paths only)...")
        all_risks = [s['risk'] for s in shortest_segments + safest_segments]
        min_risk = min(all_risks) if all_risks else 0
        max_risk = max(all_risks) if all_risks else 1
    
    mid_risk = (min_risk + max_risk) / 2
    print(f"Risk range: {min_risk:.2f} - {mid_risk:.2f} - {max_risk:.2f}")
    
    # Add SAFEST path FIRST (so it's rendered underneath)
    # This way shortest path will be on top and visible
    print("\nAdding safest path segments to map (bold dashed line, underneath)...")
    for seg in safest_segments:
        color = hex_color_from_risk(seg['risk'], min_risk, max_risk)
        
        folium.PolyLine(
            seg['coords'],
            color=color,
            weight=5,
            opacity=0.85,
            dash_array='15, 8',  # More visible dashes: 15px dash, 8px gap
            lineCap='round',
            lineJoin='round',
            popup=f"<b>Safest Route Segment</b><br>" +
                  f"ID: {seg['segment_id']}<br>" +
                  f"Length: {seg['length_m']:.0f}m<br>" +
                  f"Risk: {seg['risk']:.3f}",
            tooltip=f"Safest: {seg['risk']:.3f}",
            z_index_offset=-5
        ).add_to(m)
    
    # Add shortest path segments (solid line, on top)
    print("Adding shortest path segments to map (solid line, on top)...")
    for seg in shortest_segments:
        color = hex_color_from_risk(seg['risk'], min_risk, max_risk)
        
        # Add colored solid line (no dashes)
        folium.PolyLine(
            seg['coords'],
            color=color,
            weight=5,
            opacity=0.95,
            lineCap='round',
            lineJoin='round',
            popup=f"<b>Shortest Route Segment</b><br>" +
                  f"ID: {seg['segment_id']}<br>" +
                  f"Length: {seg['length_m']:.0f}m<br>" +
                  f"Risk: {seg['risk']:.3f}",
            tooltip=f"Shortest: {seg['risk']:.3f}",
            z_index_offset=10
        ).add_to(m)
    
    # Add markers with custom styling
    folium.Marker(
        location=[origin_lonlat[1], origin_lonlat[0]],
        popup="<b>Start</b><br>Prenzlauer Allee 80",
        icon=folium.Icon(color='green', icon='play', prefix='fa', icon_color='white'),
        tooltip='Origin',
        z_index_offset=100
    ).add_to(m)
    
    folium.Marker(
        location=[dest_lonlat[1], dest_lonlat[0]],
        popup="<b>Destination</b><br>Bonanza Coffee",
        icon=folium.Icon(color='red', icon='stop', prefix='fa', icon_color='white'),
        tooltip='Destination',
        z_index_offset=100
    ).add_to(m)
    
    # Enhanced legend with risk scale and concrete numbers
    low_val = min_risk
    mid_val = (min_risk + max_risk) / 2
    high_val = max_risk
    
    scale_label = "RELATIVE (selected paths)" if risk_scale == "relative" else "OVERALL (all segments)"
    
    legend_html = f'''
    <div style="position: fixed; bottom: 50px; right: 50px; width: 360px; height: 420px; 
                background-color: white; border:3px solid #444; z-index:9999; font-size:12px; padding: 14px;
                border-radius: 8px; box-shadow: 0 0 20px rgba(0,0,0,0.3); font-family: 'Segoe UI', Arial;">
        <p style="margin: 0 0 12px 0; font-weight: bold; font-size: 14px; color: #333;">🛣️ Route Visualization</p>
        <hr style="margin: 8px 0; border: 1px solid #ddd;">
        
        <p style="margin: 8px 0; font-size: 11px; font-weight: bold; color: #555;"><b>📍 Route Styles:</b></p>
        <p style="margin: 4px 0; padding: 6px; background: #f9f9f9; border-radius: 3px;">
            <svg width="80" height="12" style="vertical-align: middle;">
                <line x1="0" y1="6" x2="80" y2="6" stroke="#FF6600" stroke-width="5" stroke-dasharray="10,10" stroke-linecap="round"/>
            </svg>
            <b>Safest Route</b><br/>
            <span style="font-size: 9px; color: #666; margin-left: 12px;">Dashed line - underneath</span>
        </p>
        <p style="margin: 4px 0; padding: 6px; background: #f9f9f9; border-radius: 3px;">
            <svg width="80" height="12" style="vertical-align: middle;">
                <line x1="0" y1="6" x2="80" y2="6" stroke="#FF6600" stroke-width="5" stroke-linecap="round"/>
            </svg>
            <b>Shortest Route</b><br/>
            <span style="font-size: 9px; color: #666; margin-left: 12px;">Solid line - on top</span>
        </p>
        
        <hr style="margin: 10px 0; border: 1px solid #ddd;">
        
        <p style="margin: 8px 0; font-size: 10px; font-weight: bold; color: #444; background: #ffffcc; padding: 6px; border-radius: 3px;">
            📊 Scale: <b>{scale_label}</b>
        </p>
        
        <p style="margin: 8px 0; font-size: 11px; font-weight: bold; color: #555;"><b>⚠️ Risk Level:</b><br/><span style="font-size: 10px; color: #666;">(accidents per 10,000 cyclists)</span></p>
        <p style="margin: 3px 0; padding: 4px; background: #f0f0f0; border-radius: 3px; font-size: 10px;">
            <span style="color: #00AA00; font-weight: bold; font-size: 12px;">●</span> 
            <b>Low:</b> {low_val:.2f}
        </p>
        <p style="margin: 3px 0; padding: 4px; background: #f0f0f0; border-radius: 3px; font-size: 10px;">
            <span style="color: #FFA500; font-weight: bold; font-size: 12px;">●</span> 
            <b>Medium:</b> {mid_val:.2f}
        </p>
        <p style="margin: 3px 0; padding: 4px; background: #f0f0f0; border-radius: 3px; font-size: 10px;">
            <span style="color: #FF0000; font-weight: bold; font-size: 12px;">●</span> 
            <b>High:</b> {high_val:.2f}
        </p>
        
        <hr style="margin: 10px 0; border: 1px solid #ddd;">
        <p style="margin: 4px 0; font-size: 9px; color: #888; font-style: italic;">
            Both routes colored by risk gradient
        </p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    if save_path:
        m.save(save_path)
        print(f"\nMap saved to: {save_path}")
    
    # Print summary
    print("\n" + "="*60)
    print("ROUTE SUMMARY")
    print("="*60)
    print(f"Shortest path: {len(shortest_segments)} segments (solid line, видна сверху)")
    print(f"  - Total length: {result['shortest_length_stats']['length_m']:.0f}m")
    print(f"  - Total risk: {result['shortest_length_stats']['seg_risk_sum']:.2f}")
    print(f"  - Avg risk per segment: {result['shortest_length_stats']['seg_risk_sum']/len(shortest_segments):.4f}")
    
    print(f"\nSafest path: {len(safest_segments)} segments (dashed 10,10, видна снизу)")
    print(f"  - Total length: {result['constrained_min_risk_stats']['length_m']:.0f}m")
    print(f"  - Total risk: {result['constrained_min_risk_stats']['seg_risk_sum']:.2f}")
    print(f"  - Avg risk per segment: {result['constrained_min_risk_stats']['seg_risk_sum']/len(safest_segments):.4f}")
    
    return m


# Run complete visualization with risk segments
print("="*60)
print("CREATING DETAILED MAP WITH RISK-COLORED SEGMENTS")
print("="*60)

artifacts_temp = build_graph_with_costs_for_month(
    segments_df,
    2023, 6,
    crossings_gdf=crossings_gdf,
    junction_panel_gdf=junction_df,
    graph_cfg=GraphBuildConfig(metric_epsg=32633),
    cost_cfg=CostConfig(alpha=1.0, beta=1.0, gamma=0.2),
    node_snap_m=20.0,
)

result_berlin_segments = run_one_od_routing(
    segments_panel_gdf=segments_df,
    crossings_gdf=crossings_gdf,
    junction_panel_gdf=junction_df,
    year=2023, month=6,
    origin_lonlat=(13.4037, 52.5365),
    dest_lonlat=(13.393983, 52.518106),
    eps=0.10,
    alpha=1.0, beta=1.0, gamma=0.2,
)
result_berlin_segments["graph"] = artifacts_temp.G

print("\nBuilding visualization with RELATIVE risk scale...")
map_with_risks = visualize_routes_with_risk_segments(
    result_berlin_segments,
    origin_lonlat=(13.4037, 52.5365),
    dest_lonlat=(13.393983, 52.518106),
    zoom_start=14,
    save_path='route_map_with_segment_risks.html',
    risk_scale="relative"
)

map_save_path = output_dir / "plot_risk_for_one_Berlin_route_relative.html"
map_with_risks.save(map_save_path)
print(f"Map saved to {map_save_path}")

# Example 2: Using OVERALL risk scale based on all segments
print("\n" + "="*60)
print("CREATING MAP WITH OVERALL RISK SCALE")
print("="*60)

print("\nBuilding visualization with OVERALL risk scale...")
map_with_risks_overall = visualize_routes_with_risk_segments(
    result_berlin_segments,
    origin_lonlat=(13.4037, 52.5365),
    dest_lonlat=(13.393983, 52.518106),
    zoom_start=14,
    save_path='route_map_with_segment_risks_overall.html',
    risk_scale="overall",
    segments_gdf=segments_df,
    risk_col="risk_accidents_per_10k_trips"
)

map_save_path_overall = output_dir / "plot_risk_for_one_Berlin_route_overall.html"
map_with_risks_overall.save(map_save_path_overall)
print(f"Map saved to {map_save_path_overall}")


map_with_risks

